In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16, VGG19, InceptionV3
from keras.layers import Input,Dense,Flatten,UpSampling2D
from keras.models import Model
from keras.callbacks import ReduceLROnPlateau,EarlyStopping
import matplotlib.pyplot as plt
from sklearn.ensemble import VotingClassifier

In [ ]:
classes = ['mountain', 'street', 'buildings', 'sea', 'forest', 'glacier']
class_counts = []
for i in classes:
    dirpaths, dirnames, files = next(os.walk("../input/intel-image-classification/seg_train/seg_train/{name}".format(name=i)))
    file_count = len(files)
    class_counts.append(file_count)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(classes,class_counts)
plt.show()

In [ ]:
##Data Preprocessing Training Set
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2
        #vertical_flip=True
    )


train = train_datagen.flow_from_directory(
        '/kaggle/input/intel-image-classification/seg_train/seg_train/',
        target_size=(150,150),
        batch_size=32, 
        class_mode='categorical')


##Data Preprocessing Test Set
test_datagen = ImageDataGenerator(rescale=1./255) 
test = test_datagen.flow_from_directory(
        '/kaggle/input/intel-image-classification/seg_test/seg_test',
        target_size=(150,150),
        batch_size=32,
        class_mode='categorical')

In [ ]:
labels=train.class_indices
labels

**Adding Models(VGG16,VGG19,Inceptionv3**

In [ ]:
input_shape=(150,150,3)
num_classes = 6
epochs = 25

In [ ]:
def VGG_16(input_shape=input_shape, num_classes=num_classes):
    i=Input(shape=input_shape)
    x=VGG16(weights = "imagenet", include_top = False, input_shape=input_shape)(i)
    x=Flatten()(x)
    x=Dense(num_classes, activation='softmax')(x)
    model=Model(i, x)
    model.compile(loss='categorical_crossentropy', 
                  optimizer='sgd', 
                  metrics=['accuracy'])    
    print(model.summary())
    
    return model 

In [ ]:
def VGG_19(input_shape=input_shape, num_classes=num_classes):
    i=Input(shape=input_shape)
    x=VGG19(weights = "imagenet", include_top = False, input_shape=input_shape)(i)
    x=Flatten()(x)
    x=Dense(num_classes, activation='softmax')(x)
    model=Model(i, x)
    model.compile(loss='categorical_crossentropy', 
                  optimizer='sgd', 
                  metrics=['accuracy'])    
    print(model.summary())
    return model 

In [ ]:
def Inception_v3(input_shape=input_shape, num_classes=num_classes):
    i=Input(shape=input_shape)
    x=InceptionV3(weights = "imagenet", include_top = False, input_shape=input_shape)(i)
    x=Flatten()(x)
    x=Dense(num_classes, activation='softmax')(x)
    model=Model(i, x)
    model.compile(loss='categorical_crossentropy', 
                  optimizer='sgd', 
                  metrics=['accuracy'])    
    print(model.summary())
    return model 

In [ ]:
model = []
model.append(VGG_16())
model.append(VGG_19())
model.append(Inception_v3())

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.05, patience=5, min_lr=0.000002) #Optimizing Learning Rate
early_stop = EarlyStopping(monitor='val_accuracy', patience=5)

In [ ]:
#history=model[0].fit(x=training_set,validation_data=test_set,epochs=2,callbacks=[reduce_lr, early_stop],use_multiprocessing=True,workers=8)

models = []
for i in range(len(model)):
    r=model[i].fit(x=train,validation_data=test,epochs=epochs,callbacks=[reduce_lr, early_stop],use_multiprocessing=True,workers=8)
    models.append(model[i])
    model[i].save_weights("model{name}.h5".format(name=str(i)))
    plt.plot(r.history['loss'],label='loss')
    plt.plot(r.history['val_loss'],label='val_loss')
    plt.title("model{name}.h5".format(name=str(i)))
    plt.legend()
    

In [ ]:
# Predict labels with models
labels = []
for m in models:
    predicts = np.argmax(m.predict(test_set), axis=1)
    labels.append(predicts)
    
# Ensemble with voting
labels = np.array(labels)
labels = np.transpose(labels,(1,0))

In [ ]:
labels

In [ ]:
labels = stats.mode(labels, axis=1)[0]

In [ ]:
labels =  np.squeeze(labels)

In [ ]:
prediction= pd.DataFrame(labels,columns =['Ensemble_Preds'])
prediction